In [ ]:
%load_ext autoreload
%autoreload 2



%load_ext line_profiler

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm
from evalset.evalset import test_funcs, icml2016_tests
import sys, inspect

import swarm
import optimizer

In [ ]:
Adjiman = test_funcs.Adjiman()

In [ ]:
constraints = np.array(Adjiman.bounds)

In [ ]:
init_swarm = swarm.Swarm(Adjiman.do_evaluate, 10, 2, constraints)

In [ ]:
positions[0,:]=np.array([-12, 140])
positions[2,:]=np.array([-17, 22])

In [ ]:
bool_below = positions < constraints[:, 0]
bool_above = positions > constraints[:, 1]

for dim in range(init_swarm.dim):
    positions[bool_below[:, dim], dim] = constraints[dim, 0]
    positions[bool_above[:, dim], dim] = constraints[dim, 1]

    velocity[bool_below[:, dim], dim] = 0
    velocity[bool_above[:, dim], dim] = 0

In [ ]:
def rosenbrock(x):
    return (1 - x[:, 0])**2 + 100*(x[:, 1] - x[:, 0]**2)**2

In [ ]:
def ackley(x):
    f = -20*np.exp(-0.2*np.sqrt(0.5*((x[:,0]**2)+(x[:,1]**2))))-np.exp(0.5*(np.cos(2*np.pi*x[:,0])+np.cos(2*np.pi*x[:,1])))+np.exp(1)+20
    return f

In [ ]:
def print_classes():
    List = []
    for name, obj in inspect.getmembers(test_funcs):
        if inspect.isclass(obj):
            List.append(obj)
    return List

In [ ]:
icml = icml2016_tests.tests_for_nonparametric

In [ ]:
icml = icml2016_tests.tests_for_nonparametric
for idx, test in enumerate(icml):

    test_function_class = getattr(test_funcs, test['name'])
    test_function_base = test_function_class(test['dim'])
    print("#####################################################################################")
    print("Testfunction:", test['name'])
    print("Testfunc.dim:",test_function_base.dim)
    
    try:   
        opt = optimizer.Optimizer(test_function_base.do_evaluate, 60, test_function_base.dim, np.array(test_function_base.bounds), 500)
        result = opt.optimize()
        plt.plot(result['gbest_list'])
        plt.ylim(ymin=0)
        plt.show()
        print("Bounds:",test_function_base.bounds)
        print(f"The expected minimum was found at {result['x_opt']} with a functionvalue of {result['func_opt']}")
        print(f"The real minimum is located at {test_function_base.min_loc} with a functionvalue of {test_function_base.fmin}")
        print(f"Error -> {np.linalg.norm(result['x_opt']-test_function_base.min_loc)}")
    except (IndexError, ValueError):
        print("Our optimazier couldn't handel the objectiv function, trying the next one.")
    
    

In [ ]:
Plateau = test_funcs.Plateau()

In [ ]:
opt = optimizer.Optimizer(Plateau.do_evaluate, 20, Plateau.dim, np.array(Plateau.bounds), 50)

# %lprun -f optimizer.Optimizer.optimize gbest_list = opt.optimize()
# %timeit gbest_list = opt.optimize()

result = opt.optimize()
plt.plot(result['gbest_list'])
print(result['x_opt'], result['func_opt'])

In [ ]:
opt = optimizer.Optimizer(Ackley.do_evaluate, 20, Ackley.dim, np.array(Ackley.bounds), 50)

# %lprun -f optimizer.Optimizer.optimize gbest_list = opt.optimize()
# %timeit gbest_list = opt.optimize()

result = opt.optimize()
plt.plot(result['gbest_list'])
print(result['x_opt'], result['func_opt'])

In [ ]:
sw = swarm.Swarm(f, 10, 2, constraints)
# gbest_list = []
# for i in tqdm(range(100)):
#     sw.update_swarm()
#     #print(sw.pbest_position)
#     gbest, gbest_position = sw.compute_gbest()
#     print(gbest)
#     gbest_list.append(gbest)
    
gbest, gbest_position = sw.compute_gbest()
(gbest, gbest_position)

In [ ]:
lbest, lbest_position = sw.compute_lbest()

pbest = sw.pbest
pbest_position = sw.pbest_position

position = sw.position + sw.velocity
values = sw.func(position)

In [ ]:
c_1, c_2 = np.ones(2)*0.5 + np.log(2)

U_1 = np.random.uniform(0, 1, (sw.n_particles, sw.dim))
U_2 = np.random.uniform(0, 1, (sw.n_particles, sw.dim))

proj_pbest = position + c_1 * U_1 * (pbest_position - position)
proj_lbest = position + c_2 * U_2 * (lbest_position - position) 

center = (position + proj_pbest + proj_lbest) / 3
r = np.linalg.norm(center - position, axis=1)

In [ ]:
u = np.random.normal(0, 1, (sw.n_particles, sw.dim+2))
u_norm = np.linalg.norm(u, axis=1)
u /= u_norm[:,None]
u *= r[:,None]

update = u[:,:sw.dim]
new_point = center + update

# Function for hypersphere sampling

In [ ]:
def get_random_sample(r, dim):
    u = np.random.normal(0, 1, (r.shape[0], dim+2))
    u_norm = np.linalg.norm(u, axis=1)
    u /= u_norm[:,None]
    u *= r[:,None]
    return u[:,:dim]

# Test for Hypersphere sampling in 2D

In [ ]:
random_samples=[]
    
update = get_random_sample(r, sw.dim)

random_samples.append(center + update)

random_samples = np.array(random_samples)

In [ ]:
i=1
plt.plot(position[0, 0], position[0, 1], label='starting pos', marker='+', color='blue')
plt.plot(proj_pbest[0, 0], proj_pbest[0, 1], label='Proj_pbest', marker='+', color='red')
plt.plot(proj_lbest[0, 0], proj_lbest[0, 1], label='Proj_lbest', marker='o', color='green')
plt.plot(center[0, 0], center[0, 1], label='Center', marker='o', color='yellow')


# for idx, sample in enumerate(random_samples):

plt.plot(random_samples[:, 0, 0], random_samples[:, 0, 1], linestyle='None', marker='.', color='black')

plt.legend()


circle1 = plt.Circle(center[0, :] , r[0], color='purple')
plt.gca().add_patch(circle1)

plt.axis('equal')
plt.show()

# Test for Hypersphere sampling in 3D

In [ ]:
center=np.array([0, 0, 0])

In [ ]:
radius = 0.6
dim = 3

In [ ]:
# def get_random_sample(r, dim):
#     u = np.random.normal(0, 1, dim+2)
#     u_norm = np.linalg.norm(u)
#     u /= u_norm
#     u *= r
#     return u[:dim]

# Just for the 3D Plot

In [ ]:
data = [get_random_sample(radius, 3) for _ in range(100)]

In [ ]:
data = np.array(data)

In [ ]:
import plotly.express as px

In [ ]:
fig = px.scatter_3d(x=data[:, 0], y=data[:, 1], z=data[:, 2])
fig.show()

# Testfunction

Where "update" is the hypersperical sample and r is the radius of the ball in which we would like to perform a uniform draw. 

In [ ]:
update = get_random_sample(r, sw.dim)

In [ ]:
np.testing.assert_array_less(np.linalg.norm(update, axis=1), r)